# TRAINING I3W


# A) Create Envorinment, Vehicles etc

### General Parameter

In [1]:
# Define horizon as a variable to ensure consistent use across notebook (length of one rollout)
HORIZON=500                                 #103 max Horizon, wenn es vor verlassen abbrechen soll!, default war 500

# name of the experiment
experiment_name = "IntersectionExample"

# scenario class
import flow.scenarios as scenarios
print("Available scenarios:")
print(scenarios.__all__)
scenario_name = "IntersectionTWScenario"

# environment class
import flow.multiagent_envs as flowenvs
print("\nAvailable environments:")
print(flowenvs.__all__)
env_name = "MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit"

Available scenarios:
['Scenario', 'BayBridgeScenario', 'BayBridgeTollScenario', 'BottleneckScenario', 'Figure8Scenario', 'SimpleGridScenario', 'HighwayScenario', 'LoopScenario', 'MergeScenario', 'TwoLoopsOneMergingScenario', 'MultiLoopScenario', 'IntersectionScenarioTW']

Available environments:
['MultiEnv', 'MultiAgentAccelEnv', 'MultiWaveAttenuationPOEnv', 'MultiAgentIntersectionEnv', 'MultiAgentTeamSpiritIntersectionEnv', 'MultiAgentIntersectionEnv_baseline_1', 'MultiAgentIntersectionEnv_baseline_2', 'MultiAgentIntersectionEnv_baseline_3', 'MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit']


### Net Parameter

In [2]:
from flow.core.params import NetParams
from flow.scenarios.intersection import ADDITIONAL_NET_PARAMS

additionalNetParams = {
            "edge_length": 40,
            "lanes": 1,
            "speed_limit": 30
        }

net_params = NetParams( no_internal_links=False,                  #default: True   !! damit Kreuzungen nicht überspr. werden
                        inflows=None,                             #default: None
                        osm_path=None,                            #default: None
                        netfile=None,                             #default: None
                        additional_params=additionalNetParams     #default: None   !!
                      )

### InitialConfig Parameter

In [3]:
from flow.core.params import InitialConfig

initial_config = InitialConfig( shuffle=True,                            #default: False         !!
                                spacing="custom",                        #default: "uniform"     !!
                                min_gap=10,                              #default: 0
                                perturbation=29.99,                      #default: 0.0            !!        
                                x0=0,                                    #default: 0
                                bunching=0,                              #default: 0
                                lanes_distribution=float("inf"),         #default: float("inf")
                                edges_distribution="all",                #default: "all"
                                additional_params=None )                 #default: None

### SUMO Parameter

In [4]:
from flow.core.params import SumoParams

sumo_params = SumoParams( port = None,                  #default: None
                          sim_step=0.1,                 #default: 0.1
                          emission_path=None,           #default: None
                          lateral_resolution=None,      #default: None
                          no_step_log=True,             #default: True
                          render=False,                 #default: False
                          save_render=False,            #default: False
                          sight_radius=25,              #default: 25
                          show_radius=False,            #default: False
                          pxpm=2,                       #default: 2
                          overtake_right=False,         #default: False    
                          seed=None,                    #default: None
                          restart_instance=False,       #default: False
                          print_warnings=True,          #default: True
                          teleport_time=-1,             #default: -1
                          num_clients=1,                #default: 1
                          sumo_binary=None )            #default: None

### Environment Parameter

In [5]:
from flow.core.params import EnvParams

additionalEnvParams = {
        # maximum acceleration of autonomous vehicles
        "max_accel": 3,
        # maximum deceleration of autonomous vehicles
        "max_decel": 3,
        "target_velocity": 30
    }

env_params = EnvParams( additional_params=additionalEnvParams, #default: None    !!
                        horizon=HORIZON,                       #default: 500     !!
                        warmup_steps=0,                        #default: 0       
                        sims_per_step=1,                       #default: 1
                        evaluate=False )                       #default: False

### Vehicles Parameter

In [6]:
from flow.core.params import VehicleParams

# import vehicles dynamics models
#from flow.controllers import SumoCarFollowingController
from flow.controllers import ContinuousRouter
#from flow.controllers.lane_change_controllers import SumoLaneChangeController
from flow.controllers.lane_change_controllers import StaticLaneChanger
from flow.controllers import RLController
from flow.core.params import SumoLaneChangeParams
from flow.core.params import SumoCarFollowingParams
from random import *

vehicles = VehicleParams()

#### Add RL-Agent controlled vehicles 

In [7]:
# car following parameters, default: None
cf_parameter = SumoCarFollowingParams(
                speed_mode="aggressive")
# lane change parameters, default: None
lc_parameter =  None

vehicles.add( # name of the vehicle
                veh_id = "rl",
              # acceleration controller, default: (SumoCarFollowingController, {})
                acceleration_controller=(RLController, {}),
              # lane_change_controller, default: (SumoLaneChangeController, {})
                lane_change_controller=(StaticLaneChanger,{}),
              # routing controller, default: None
                routing_controller=(ContinuousRouter, {}),
              # initial speed, default: 0
                initial_speed=0,
              # number of vehicles, default: 1 
                num_vehicles=2,
                
                car_following_params=cf_parameter
              # speed mode, default: "right_of_way"
                #speed_mode="aggressive",
              # lane change mode, default: "no_lat_collide"
                #lane_change_mode="aggressive", 
              # car following parameter, default: None
                #sumo_car_following_params=cf_parameter,
              # lane change parameter, default: None
                #sumo_lc_params=lc_parameter
)

### Flow Parameter

In [8]:
# Creating flow_params. Make sure the dictionary keys are as specified. 
flow_params = dict( # name of the experiment
                      exp_tag=experiment_name,
                    # name of the flow environment the experiment is running on
                      env_name=env_name,
                    # name of the scenario class the experiment uses
                      scenario=scenario_name,
                    # simulator that is used by the experiment
                      simulator='traci',
                    # sumo-related parameters (see flow.core.params.SumoParams)
                      sim=sumo_params,
                    # environment related parameters (see flow.core.params.EnvParams)
                      env=env_params,
                    # network-related parameters (see flow.core.params.NetParams and
                    # the scenario's documentation or ADDITIONAL_NET_PARAMS component)
                      net=net_params,
                    # vehicles to be placed in the network at the start of a rollout 
                    # (see flow.core.vehicles.Vehicles)
                      veh=vehicles,
                   # (optional) parameters affecting the positioning of vehicles upon 
                   # initialization/reset (see flow.core.params.InitialConfig)
                      initial=initial_config
                )

# B) Training

In [9]:
import json

import ray
try:
    from ray.rllib.agents.agent import get_agent_class
except ImportError:
    from ray.rllib.agents.registry import get_agent_class
from ray.tune import run_experiments
from ray.tune.registry import register_env

from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder

from ray import tune
from ray.rllib.agents.ppo.ppo_policy_graph import PPOPolicyGraph

In [10]:
# number of parallel workers
N_CPUS = 2
# number of rollouts per training iteration
N_ROLLOUTS = 20

ray.init(redirect_output=True, num_cpus=N_CPUS+1)

Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-03-27_18-49-37_3993/logs.
Waiting for redis server at 127.0.0.1:18844 to respond...
Waiting for redis server at 127.0.0.1:41057 to respond...
Starting the Plasma object store with 6.554658406 GB memory using /dev/shm.

View the web UI at http://localhost:8889/notebooks/ray_ui.ipynb?token=0909cc23ad45c470113c14675027e7445175df12c7b32da8



{'node_ip_address': '192.168.2.102',
 'object_store_addresses': ['/tmp/ray/session_2019-03-27_18-49-37_3993/sockets/plasma_store'],
 'raylet_socket_names': ['/tmp/ray/session_2019-03-27_18-49-37_3993/sockets/raylet'],
 'redis_address': '192.168.2.102:18844',
 'webui_url': 'http://localhost:8889/notebooks/ray_ui.ipynb?token=0909cc23ad45c470113c14675027e7445175df12c7b32da8'}

In [11]:
# The algorithm or model to train. This may refer to "
#      "the name of a built-on algorithm (e.g. RLLib's DQN "
#      "or PPO), or a user-defined trainable function or "
#      "class registered in the tune registry.")
alg_run = "PPO"

agent_cls = get_agent_class(alg_run)
config = agent_cls._default_config.copy()
config["num_workers"] = N_CPUS  # number of parallel workers
config["train_batch_size"] = HORIZON * N_ROLLOUTS  # batch size
config["gamma"] = 0.999  # discount rate default 0.999
config["model"].update({"fcnet_hiddens": [100, 50, 25]})  # size of hidden layers in network defaule 64 32
config["use_gae"] = True  # using generalized advantage estimation
config["lambda"] = 0.97  
#config["sgd_minibatch_size"] = min(16 * 1024, config["train_batch_size"])  # stochastic gradient descent
#config["sample_batch_size"] = config["train_batch_size"]/config["num_workers"] # 200 default, trotzdem zu hoch?
config["kl_target"] = 0.02  # target KL divergence
config["num_sgd_iter"] = 10  # number of SGD iterations
config["horizon"] = HORIZON  # rollout horizon

# save the flow params for replay
flow_json = json.dumps(flow_params, cls=FlowParamsEncoder, sort_keys=True,
                       indent=4)  # generating a string version of flow_params
config['env_config']['flow_params'] = flow_json  # adding the flow_params to config dict
config['env_config']['run'] = alg_run

# Call the utility function make_create_env to be able to 
# register the Flow env for this experiment
create_env, gym_name = make_create_env(params=flow_params, version=0)

# Register as rllib env with Gym
register_env(gym_name, create_env)

In [12]:
# multi agent policy mapping
test_env = create_env()
obs_space = test_env.observation_space
act_space = test_env.action_space

def gen_policy():
    return (PPOPolicyGraph, obs_space, act_space, {})

# Setup PG with an ensemble of `num_policies` different policy graphs
policy_graphs = {'rl_0': gen_policy()}
    
def policy_mapping_fn(agent_id):
    return 'rl_0'

config.update({
        'multiagent': {
            'policy_graphs': policy_graphs,
            'policy_mapping_fn': tune.function(policy_mapping_fn),
            'policies_to_train': ['rl_0']
        }
    })

 Starting SUMO on port 44139


20.553154398181952
18.504479839823905


In [13]:
trials = run_experiments({
    flow_params["exp_tag"]: {
        "run": alg_run,  # RL algorithm to run
        "env": gym_name,  # environment name generated earlier
        "config": {  # configuration params (must match "run" value)
            **config
        },
        "checkpoint_freq": 1,  # number of iterations between checkpoints
        "max_failures": 999,
        "stop": {  # stopping conditions
            "training_iteration": 1000,  # number of iterations to stop after
        },
    },
})

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/3 CPUs, 0/1 GPUs
Memory usage on this node: 2.5/16.4 GB

Created LogSyncer for /home/thorsten/ray_results/IntersectionExample/PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0_2019-03-27_18-49-39vs4d3yjp -> 
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 2.5/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:	RUNNING

Result for PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:
  custom_metrics: {}
  date: 2019-03-27_18-50-34
  done: false
  episode_len_mean: 436.27272727272725
  episode_reward_max: 426.82213129389606
  episode_reward_mean: 106.97591505808391
  episode_reward_min: -115.22056416182052
  episodes_this_iter: 22
  episodes_total: 22
  experiment_id: f959ca97c66b498388a02f90a5450ca0
  hostname: Gandalf
  info:
    grad_time_ms:

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 3.4/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:	RUNNING [pid=4068], 117 s, 5 iter, 50000 ts, 402 rew

Result for PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:
  custom_metrics: {}
  date: 2019-03-27_18-52-28
  done: false
  episode_len_mean: 318.89
  episode_reward_max: 704.4184270418435
  episode_reward_mean: 467.9996323150662
  episode_reward_min: -124.29876912235675
  episodes_this_iter: 36
  episodes_total: 166
  experiment_id: f959ca97c66b498388a02f90a5450ca0
  hostname: Gandalf
  info:
    grad_time_ms: 3991.273
    load_time_ms: 9.38
    num_steps_sampled: 60000
    num_steps_trained: 60000
    rl_0:
      cur_kl_coeff: 0.006250001490116119
      cur_lr: 4.999999873689376e-05
      entropy: 1.4083789587020874
      kl: 0.0065506016835570335
      poli

Result for PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:
  custom_metrics: {}
  date: 2019-03-27_18-54-24
  done: false
  episode_len_mean: 195.31
  episode_reward_max: 722.2168823420557
  episode_reward_mean: 429.1218085063419
  episode_reward_min: -133.64956861302542
  episodes_this_iter: 52
  episodes_total: 397
  experiment_id: f959ca97c66b498388a02f90a5450ca0
  hostname: Gandalf
  info:
    grad_time_ms: 3896.224
    load_time_ms: 1.58
    num_steps_sampled: 110000
    num_steps_trained: 110000
    rl_0:
      cur_kl_coeff: 0.00039062509313225746
      cur_lr: 4.999999873689376e-05
      entropy: 1.41211998462677
      kl: 0.004875446669757366
      policy_loss: -0.00132059957832098
      total_loss: 1263.8843994140625
      vf_explained_var: 0.06937707215547562
      vf_loss: 1263.8858642578125
    sample_time_ms: 19023.591
    update_time_ms: 5.947
  iterations_since_restore: 11
  node_ip: 192.168.2.102
  num_metric_batches_dropped: 0
  pid: 4068
  policy_reward_me

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 3.4/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:	RUNNING [pid=4068], 372 s, 16 iter, 160000 ts, 465 rew

Result for PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:
  custom_metrics: {}
  date: 2019-03-27_18-56-47
  done: false
  episode_len_mean: 167.29
  episode_reward_max: 717.0755032543003
  episode_reward_mean: 474.66708026984185
  episode_reward_min: -112.35038403251906
  episodes_this_iter: 60
  episodes_total: 752
  experiment_id: f959ca97c66b498388a02f90a5450ca0
  hostname: Gandalf
  info:
    grad_time_ms: 3972.516
    load_time_ms: 1.748
    num_steps_sampled: 170000
    num_steps_trained: 170000
    rl_0:
      cur_kl_coeff: 6.103517080191523e-06
      cur_lr: 4.999999873689376e-05
      entropy: 1.3909573554992676
      kl: 0.004349222406744957
    

Result for PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:
  custom_metrics: {}
  date: 2019-03-27_18-58-55
  done: false
  episode_len_mean: 133.63
  episode_reward_max: 707.2048169168594
  episode_reward_mean: 395.8934197377254
  episode_reward_min: -133.8188883367481
  episodes_this_iter: 77
  episodes_total: 1101
  experiment_id: f959ca97c66b498388a02f90a5450ca0
  hostname: Gandalf
  info:
    grad_time_ms: 4162.892
    load_time_ms: 1.773
    num_steps_sampled: 220000
    num_steps_trained: 220000
    rl_0:
      cur_kl_coeff: 1.907349087559851e-07
      cur_lr: 4.999999873689376e-05
      entropy: 1.379974603652954
      kl: 0.004213887732475996
      policy_loss: -0.0016978237545117736
      total_loss: 2292.095458984375
      vf_explained_var: 0.10830944031476974
      vf_loss: 2292.096923828125
    sample_time_ms: 20580.121
    update_time_ms: 5.369
  iterations_since_restore: 22
  node_ip: 192.168.2.102
  num_metric_batches_dropped: 0
  pid: 4068
  policy_reward_m

...

Result for PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:
  custom_metrics: {}
  date: 2019-03-28_01-30-17
  done: false
  episode_len_mean: 132.29
  episode_reward_max: 788.7568782821708
  episode_reward_mean: 698.9765775681019
  episode_reward_min: -133.8693519431127
  episodes_this_iter: 78
  episodes_total: 73270
  experiment_id: f959ca97c66b498388a02f90a5450ca0
  hostname: Gandalf
  info:
    grad_time_ms: 3713.785
    load_time_ms: 1.567
    num_steps_sampled: 9790000
    num_steps_trained: 9790000
    rl_0:
      cur_kl_coeff: 0.01912524551153183
      cur_lr: 4.999999873689376e-05
      entropy: 0.8439419865608215
      kl: 0.01727464608848095
      policy_loss: -0.0034659577067941427
      total_loss: 177.73648071289062
      vf_explained_var: 0.9876263737678528
      vf_loss: 177.73963928222656
    sample_time_ms: 17916.586
    update_time_ms: 5.287
  iterations_since_restore: 979
  node_ip: 192.168.2.102
  num_metric_batches_dropped: 0
  pid: 4068
  policy_rewar

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 7.5/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:	RUNNING [pid=4068], 24077 s, 984 iter, 9840000 ts, 710 rew

Result for PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:
  custom_metrics: {}
  date: 2019-03-28_01-32-29
  done: false
  episode_len_mean: 132.36
  episode_reward_max: 794.6099575894347
  episode_reward_mean: 717.421872994941
  episode_reward_min: -60.74083524538912
  episodes_this_iter: 78
  episodes_total: 73733
  experiment_id: f959ca97c66b498388a02f90a5450ca0
  hostname: Gandalf
  info:
    grad_time_ms: 3704.193
    load_time_ms: 1.52
    num_steps_sampled: 9850000
    num_steps_trained: 9850000
    rl_0:
      cur_kl_coeff: 0.01912524551153183
      cur_lr: 4.999999873689376e-05
      entropy: 0.7819920778274536
      kl: 0.03739117830991745
   

Result for PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:
  custom_metrics: {}
  date: 2019-03-28_01-34-20
  done: false
  episode_len_mean: 127.92
  episode_reward_max: 787.8460850504433
  episode_reward_mean: 697.5877244614986
  episode_reward_min: -89.7640772517754
  episodes_this_iter: 79
  episodes_total: 74119
  experiment_id: f959ca97c66b498388a02f90a5450ca0
  hostname: Gandalf
  info:
    grad_time_ms: 3714.687
    load_time_ms: 1.53
    num_steps_sampled: 9900000
    num_steps_trained: 9900000
    rl_0:
      cur_kl_coeff: 0.01912524551153183
      cur_lr: 4.999999873689376e-05
      entropy: 0.8064212203025818
      kl: 0.013590800575911999
      policy_loss: -0.0010026206728070974
      total_loss: 346.4015197753906
      vf_explained_var: 0.9702374339103699
      vf_loss: 346.4022521972656
    sample_time_ms: 18346.264
    update_time_ms: 6.031
  iterations_since_restore: 990
  node_ip: 192.168.2.102
  num_metric_batches_dropped: 0
  pid: 4068
  policy_reward_m

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 7.6/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:	RUNNING [pid=4068], 24322 s, 995 iter, 9950000 ts, 703 rew

Result for PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:
  custom_metrics: {}
  date: 2019-03-28_01-36-34
  done: false
  episode_len_mean: 136.16
  episode_reward_max: 791.6800608931242
  episode_reward_mean: 717.5784267215184
  episode_reward_min: 155.99720865512734
  episodes_this_iter: 78
  episodes_total: 74583
  experiment_id: f959ca97c66b498388a02f90a5450ca0
  hostname: Gandalf
  info:
    grad_time_ms: 3756.84
    load_time_ms: 1.548
    num_steps_sampled: 9960000
    num_steps_trained: 9960000
    rl_0:
      cur_kl_coeff: 0.028687868267297745
      cur_lr: 4.999999873689376e-05
      entropy: 0.7913187742233276
      kl: 0.02261200360953808
 

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/3 CPUs, 0/1 GPUs
Memory usage on this node: 7.6/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
TERMINATED trials:
 - PPO_MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit-v0_0:	TERMINATED [pid=4068], 24433 s, 1000 iter, 10000000 ts, 698 rew

